# Day 2 – Tutorial 01: Phylogenetic Analysis using biopython and RAxML

## Theoretical Aspects 

Phylogenetic analysis lies at the core of genomics and bioinformatics and seeks to establish the evolutionary relationships between different homologous DNA or protein sequences and their ancestral sequences (common ancestors) from which they emerge. A typical result from a phylogenetic analysis is a **phylogenetic tree**, such as the following:

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/phylo_01.png' width=500/>
<figcaption>FIGURE 1. A phylogenetic tree visually represents a hypothesis of how a group of sequences
are related. This figure explores how the way a tree is drawn conveys information. This tree shows how the seven sequences at the tips of the branches, called taxa, are related. Each horizontal branch represents an evolutionary
lineage. The length of the branch is arbitrary
unless the diagram specifies that branch lengths
represent information such as time or amount of
genetic change. Each branch point represents the common ancestor of the evolutionary lineages diverging from it. One of these branch points represents the common ancestor of all the sequences shown in this tree.
<br>Urry LA et al (2020). Campbell Biology. <i>Pearson Education, 12th Ed</i></figcaption></center>
</figure>

In this representation, the tips of the tree (or the leaves) correspond to different, currently existing sequences of genes or proteins (usually called the **taxa**), thus they represent real data. The **nodes** or branch points connecting two sequences are the points of divergence in sequence evolution, namely **common ancestors** of extant taxa. The connection of two or more sequences with a hypothetical, extinct ancestral sequence allows grouping actual sequences into **clades**. On the other hand, the branch lengths represent the evolutionary changes between an ancestor and its descendant. Please note that we are saying **'sequences'** and not **'species'**, as such statement requires the assumption of genetic isolation (i.e. all sequences in a given species show the same evolutionary drifts, but we learn from lectures that we can have horizontal gene transfer, right?).

Many things can be said about this tree. First, a **phylogenetic tree** is based on a multiple sequence alignment and thus it is **as good as the alignment is**. See for example the polytomy in the ancestral node connecting taxa D, E and F; the sequence information between these sequences did not allow discerning how these sequences evolved from a common ancestor (i.e. imagine that there were two mutations and the phylogenetic analysis does not know which one comes first or second).

Second, taxon G is defined as a **basal taxon**, a lineage that diverges from all other members of its group early in the history of the group.

Lastly, the phylogenetic tree is **rooted** through an **outgroup**, i.e. a sequence (or a group or sequences) that is more distantly related to the ingroup sequences than the ingroup sequences are to each other; a distant homolog. As an outgroup sequence is not easy to define for all phylogenetic inferences, a vast number of phylogenetic trees in the literature are **unrooted** (i.e. there is no outgroup).

A phylogenetic tree is an estimation of the possible routes of evolution from ancestral lineages to present-day sequences that depends on many variables. We will smoothly go through all the steps of this inference, from sequence alignment to the “resurrection” of the most probable ancestral sequence.

## Overview

In this tutorial we will work with the family of forkhead box (Fox) transcription factors, a protein family that we currently study [Medina E et al (2016) [*Biophys J 110 (11), 2349-2360*](http://dx.doi.org/10.1016/j.bpj.2016.04.043); Medina E et (2020) [*J Mol Biol 432(19), 5411-5429*](http://dx.doi.org/10.1016/j.jmb.2020.07.017)] as part of a collaboration between the [Protein Biophysics, Biochemistry and Bioinformatics Lab](https://pb3.sitios.ing.uc.cl) from Pontificia Universidad Católica de Chile with the [Biochemistry and Molecular Biology Lab](https://sites.google.com/view/labbq) from Universidad de Chile.

These proteins are master controllers of gene expression in several eukaryotes since early *Bilateria*, and recently it has been determined that they also participate in chromosome remodeling since their structure is similar to the structure of histones. In humans, there are 19 different Fox subfamilies (from A to S), with almost all of them corresponding to monomeric proteins, except for FoxP. The members of the FoxP subfamily are known to form dimers to allow forming long-distance chromosome interactions due to a key Pro-to-Ala mutation in their sequence (see below).

<figure>
<center>
<img src='https://raw.githubusercontent.com/pb3lab/ibm3202/master/images/phylo_02.png'/>
<figcaption>FIGURE 2. Sequence determinants of the structural acrobatics of the DNA-binding domain of human FoxP transcription factors. The left panel shows how FoxP proteins oligomerize by exchanging identical secondary structure elements between adjacent subunits, a mechanism known as domain swapping. An experimentally validated key Ala residue that enables domain swapping is indicated, which has been demonstrated to be conserved in all human FoxP proteins and replaced by Pro in all monomeric Fox members, as shown in the multiple sequence alignment on the right.
<br> Medina E et al (2016) <i>Biophys J 110 (11), 2349-2360</i>; Stroud JC et al (2008) <i>Structure 14(1), 159-66</i></figcaption></center>
</figure>

Today, we will infer where did the evolutionary novelty of forming dimers emerged during the
evolution of Fox proteins using **RAxML** (Randomized Axelerated Maximum Likelihood), a popular program for phylogenetic analyses of large datasets under maximum likelihood.

#Part 0. Downloading and Installing the required software

Before we start, you must first **remember to start the hosted runtime in Google Colab**.

Then, we must install several pieces of software to perform this tutorial. Namely:
- **biopython** for searching, retrieving, parsing and storing DNA and protein sequences.
- **miniconda**, a free minimal installer of **conda** for software package and environment management.
- **MAFFT**, a multiple sequence alignment program for unix-like operating systems that offers a range of multiple alignment methods.
- **ModelTest-ng**, a tool for selecting the best-fit model of evolution for DNA and protein alignments.
- **RAxML-ng**, a phylogenetic tree inference tool which uses maximum-likelihood (ML) optimality criterion.

After several tests, the following installation instructions are the best way of setting up **Google Colab** for this laboratory session.

1. We will first install **biopython** as follows:

In [ ]:
#Installing biopython using pip
!pip install biopython

In [ ]:
#Importing biopython and os for safety
import os
import sys
import Bio

2. Finally, we will install conda to be able to install **mafft**, **cd-hit**, **RAxML-NG** and **ModelTest-NG** as follows:

In [ ]:
#Install conda using the new conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Install RAxML-NG and ModelTest-NG from
#the bioconda repository
!conda install -c bioconda raxml-ng modeltest-ng mafft cd-hit --yes

⚠️ **WARNING**: After installation of condacolab, the runtime session will crash. This is a normal behaviour, but it prevents to automatically execute all other cell codes after installation (`Runtime -> Run all` will not work).

#Part I - Retrieve Fox protein sequences using BLAST

Biopython is an excellent companion for working with DNA and protein sequences and also with structures. Here, we will show how to use it for retrieving Fox protein sequences using BLAST directly into Google Colab.

1. First, we will start by using _Entrez_ to retrieve the sequence of accession code **2KIU_A** in FASTA format and _SeqIO_ to be able to read, parse and/or write this sequence. Besides downloading our sequence, we will also store the sequence into a string (here, termed as _aaseq_) for its subsequent use in this tutorial.

In [ ]:
from Bio import SeqIO, Entrez
#Setting up your email to be able to use Entrez
Entrez.email = 'your.email@uc.cl'
#Here, we set up a temporary handle with our downloaded sequence in fasta format
temp = Entrez.efetch(db="protein",rettype="fasta",id="2KIU_A")
#Creating a fasta file to write our downloaded sequence
aaseq_out = open("2KIU_A.fasta",'w')
#Reading the sequence information as a string in fasta format
aaseq = SeqIO.read(temp, format="fasta")
#Writing the sequence record in fasta format
SeqIO.write(aaseq,aaseq_out,"fasta")
#Closing both the temp handle and the FASTA file
temp.close()
aaseq_out.close()

2. What is great about _SeqIO_ is that you can use it to manipulate your sequence (e.g. sorting, changing formats, etc) and also to print information about your sequence, such as its description, sequence and accession ID.

  You can try these commands below by first writing **"aaseq."** and then selecting one of the autocomplete options suggested by Google Colab, as exemplified below for getting the sequence of the protein

In [ ]:
#Printing the number of amino acids as an example
print("Amino acid sequence:")
print(aaseq.seq)

**💡 HINT:** If you are working with a protein/nucleic sequence of your own instead of an accession ID, you can store the sequence in a string using the _**Seq**_ function as shown below:



> ```from Bio.Seq import Seq
my_seq = Seq("AEVRPPFTYASLIRQAILESPEKQLTLNEIYNWFTRMFPYFRRNAATWKNAVRHNLSLHKYFVRVENVKGAVWTVDEVEFQKRRPQK")```

3. Once we have already stored the information of our query sequence in a string, we can use it to perform a BLAST search inside Google Colab via biopython through _NCBIWWW_.

  The code cell bellow specifies the blast program **blastp**, the database **pdb** and the query sequence. This process should not take longer than 2 min.

In [ ]:
#Using biopython to perform a blast search
%%time
from Bio.Blast import NCBIWWW
#NCBIWWW.qblast(program, database, sequence)
result_handle = NCBIWWW.qblast("blastp", "pdb", aaseq.seq, entrez_query='human[organism]')

**💡 HINT:** Please note that we are using the PDB protein database only for the purposes of this tutorial, as it has a limited number of protein sequences (due to the limited number of structures). However, you might find the use of another database more suitable for your work with your own protein or DNA sequences, as they contain a higher number of sequences, hence a higher sequence variability/redundancy.

4. In order to parse this data, we need to store it in a handle for post-processing using _NCBIXML_, as shown below:

In [ ]:
#Read the results in XML format for parsing BLAST records
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)
#This is required to reset the searches and start over again
result_handle.close()

5. We can now use all the functions in Bio.Blast to print a BLAST-like briefing of the results, and to even filter the results according to parameters such as the sequence identity, coverage and/or e-value. **Carefully examine the commands executed below to achieve this task.**

In [ ]:
#Here, we show how we can set a sequence identity cut-off
#Many other cut-offs can be employed!
PIDcut=1.00
#Printing all BLAST parameters similarly to the website
for alignment in blast_record.alignments:
  for hsp in alignment.hsps:
#Here, we add a condition to only print hits equal or less than a PID cutoff
    if(hsp.identities/hsp.align_length) <= PIDcut:
      print("Accession code:", alignment.hit_id)
      print("Sequence length:", alignment.length)
      print("Alignment length:", hsp.align_length)
      print("E-value:", hsp.expect)
#The following command calculates the sequence identity relative to
#the length of the alignment
      print("Sequence Identity [%]:", "{:.2f}".format(100*hsp.identities/hsp.align_length))
#The following command calculates the sequence coverage relative to
#the lenght of the sequence
      print("Sequence Coverage [%]:", "{:.2f}".format(100*sum(c.isalpha() for c in hsp.query)/len(aaseq)))
      print()
#Here, we print the first 60 characters of the query and hit and their matches
      print("query:", hsp.query[0:60] + "...")
      print("      ", hsp.match[0:60] + "...")
      print("sbjct:", hsp.sbjct[0:60] + "...")
      print()


**📚 HOMEWORK:** Copy the code cell above and play around with it to add other filters instead of PID, such as a minimum E-value or sequence coverage.

6. After the BLAST search and filtering, we would like to retrieve all sequences that match our selection criteria. Here, we opted to download the top 20 sequences, but you can use filters such as minimum sequence coverage, minimum e-value and maximum PID to limit your output sequences.

In [ ]:
#Setting up your email to be able to use Entrez
Entrez.email = 'your.email@uc.cl'

#Generate a loop to write all sequences into an output file
with open("sequences.fasta", "a") as allhits_out:
#Check how we are indicating to use the top 20 hits
  for alignment in blast_record.alignments[:20]:
    for hsp in alignment.hsps:
    #Here, we add a condition to print only sequences below a PID cutoff
      if(hsp.identities/len(hsp.match)) <= PIDcut:
        print("Fetching protein sequence:", alignment.hit_id)
        fetch = Entrez.efetch(db="protein", id=alignment.hit_id, rettype="fasta")
        #Reading the sequence stored in the temporary string in fasta format
        allhits_seqs = SeqIO.read(fetch, format="fasta")
        #Writing the sequence and its ID in fasta format
        SeqIO.write(allhits_seqs,allhits_out,"fasta")
        fetch.close()
#Closing the efetch and file
allhits_out.close()

😱 **EMERGENCY BACKUP!** In case BLASTP fails due to an issue with the NCBI servers, you can download a FASTA file containing 20 protein sequence homologs generated on Oct 9, 2022:

In [ ]:
!wget https://raw.githubusercontent.com/pb3lab/workshops/main/backups/saocarlos2022/D2-T1_sequences.fasta -O sequences.fasta

#Part II - Obtain and edit a Multiple Sequence Alignment (MSA) using MAFFT and biopython

Once BLAST is resolved, we can proceeed with the **Multiple Sequence Alignment (MSA)**. In this case, we will first employ **MAFFT** to align all retrieved sequences.

1. To perform a MSA alignment using MAFFT, we can again use the biopython _Bio.Align.Applications_ wrapper as shown in the code cell below.

In [ ]:
from Bio.Align.Applications import MafftCommandline
mafft_cline=MafftCommandline(input="sequences.fasta")
print(mafft_cline)
stdout, stderr = mafft_cline()
with open("aligned.fasta", "w") as handle:
  handle.write(stdout)
from Bio import AlignIO
align = AlignIO.read("aligned.fasta", "fasta")

2. The result from this algorithm, which is an extension of the **Pairwise Alignment Algorithms** we discussed during Lectures, can be seen in an online MSA viewer such as [Alignment Viewer 2.0](https://fast.alignmentviewer.org/) or using the form below.

In [ ]:
#@title Protein MSA Viewer in Google Colab
#The following code is modified from the wonderful viewer developed by Damien Farrell
#https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

#Importing all modules first
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

#Setting up the amino color code according to Zappo color scheme
def get_colors(seqs):
    #make colors for bases in sequence
    text = [i for s in list(seqs) for i in s]
    #Use Zappo color scheme
    clrs =  {'K':'red',
             'R':'red',
             'H':'red',             
             'D':'green',
             'E':'green',
             'Q':'blue',
             'N':'blue',
             'S':'blue',
             'T':'blue',
             'A':'blue',
             'I':'blue',
             'L':'blue',
             'M':'blue',
             'V':'blue',
             'F':'orange',
             'Y':'orange',
             'W':'orange',
             'C':'blue',
             'P':'yellow',
             'G':'orange',
             '-':'white'}
    colors = [clrs[i] for i in text]
    return colors

#Setting up the MSA viewer
def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]    
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None, plot_width= plot_width, plot_height=50,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False  

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, plot_width=plot_width, plot_height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    return p

#Loading the viewer by indicating the MSA file and format to read
#@markdown Name of the MSA file (including the file extension)
MSAfile = 'aligned.fasta' #@param {type:"string"}
MSAformat = 'fasta' #@param {type:"string"}
aln = AlignIO.read(MSAfile,MSAformat)
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)

3. As you can see, the sequences are aligned, as many residues that are conserved between different sequences occupy the same columns within the alignment. However, **we can also see
some sequences that are longer on the ends of the protein**. These ends will contribute nothing to the alignment, as there is nothing to compare them to. Therefore, we will trim the sequences on both N-and C-ends by selecting the regions we want to eliminate.

  Due to time restrictions, we include a script below that trims the N- and C-ends of the alignment based on finding the first and lasts columns of the MSA without any gaps.

**💡 HINT:** Ideally, you would manually correct the alignment and trim the ends of the sequences after careful inspection, which **we encourage you to do in real cases**.


In [ ]:
import sys
from Bio import AlignIO
aln = AlignIO.read("aligned.fasta", "fasta")

for fcol in range(aln.get_alignment_length()):
  if not "-" in aln[:, fcol]:
    position1 = fcol
    print("First full column is {}".format(fcol))
    break
for lcol in reversed(range(aln.get_alignment_length())):
  if not "-" in aln[:, lcol]:
    position2 = lcol+1
    print("Last full column is {}".format(lcol))
    break

print("New alignment:")
print(aln[:, position1:position2])

with open("aligned_trimmed.fasta", "w") as handle:
  count = (SeqIO.write(aln[:, position1:position2], handle, "fasta"))

trim = AlignIO.read("aligned_trimmed.fasta", "fasta")

4. We also need to **make sure that there are no duplicated sequences included in it**. Having redundant information does not have any benefit for the phylogenetic analysis that we are pursuing here.

  A suitable programs to eliminate duplicated in our alignment is **CD-HIT**, which we will employ below:

In [ ]:
#cd-hit only reads ungapped, single-line fasta, which is why we convert
#our file first using biopython as shown below
from Bio.Seq import Seq
from Bio import SeqIO
with open("file_for_cdhit.fasta", "w") as o:
    for record in SeqIO.parse("aligned_trimmed.fasta", "fasta"):
        record.seq = record.seq.ungap("-")
        SeqIO.write(record, o, "fasta-2line")

#Now we use cd-hit
!cd-hit -i file_for_cdhit.fasta -o cdhit_nodup.fasta -c 1.0

**💡 HINT:** Note that we are using `-c 1.0`, where 1.0 refers to 100% PID, to only eliminate duplicates. However, you could cluster your sequences to lower PID if required (e.g. you have too many sequences that are highly similar to each other).


5. We will generate again a multiple sequence alignment using our filtered sequences and visually inspect it.

In [ ]:
from Bio.Align.Applications import MafftCommandline
mafft_cline=MafftCommandline(input="cdhit_nodup.fasta")
print(mafft_cline)
stdout, stderr = mafft_cline()
with open("aligned_trimmed_final.fasta", "w") as handle:
  handle.write(stdout)
from Bio import AlignIO
align = AlignIO.read("aligned_trimmed_final.fasta", "fasta")

In [ ]:
#@title Protein MSA Viewer in Google Colab
#The following code is modified from the wonderful viewer developed by Damien Farrell
#https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

#Importing all modules first
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

#Setting up the amino color code according to Zappo color scheme
def get_colors(seqs):
    #make colors for bases in sequence
    text = [i for s in list(seqs) for i in s]
    #Use Zappo color scheme
    clrs =  {'K':'red',
             'R':'red',
             'H':'red',             
             'D':'green',
             'E':'green',
             'Q':'blue',
             'N':'blue',
             'S':'blue',
             'T':'blue',
             'A':'blue',
             'I':'blue',
             'L':'blue',
             'M':'blue',
             'V':'blue',
             'F':'orange',
             'Y':'orange',
             'W':'orange',
             'C':'blue',
             'P':'yellow',
             'G':'orange',
             '-':'white'}
    colors = [clrs[i] for i in text]
    return colors

#Setting up the MSA viewer
def view_alignment(aln, fontsize="9pt", plot_width=800):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]    
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>100:
        viewlen=100
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, xwheel_zoom, reset, save"

    #entire sequence view (no text, with zoom)
    p = figure(title=None, plot_width= plot_width, plot_height=50,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False  

    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, plot_width=plot_width, plot_height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                text_font="monospace",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    return p

#Loading the viewer by indicating the MSA file and format to read
#@markdown Name of the MSA file (including the filetype)
MSAfile = 'aligned_trimmed_final.fasta' #@param {type:"string"}
MSAformat = 'fasta' #@param {type:"string"}
aln = AlignIO.read(MSAfile,MSAformat)
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)

#Part III - Phylogenetic inference and ancestral sequence reconstruction using RAxML

We will use our final, trimmed and filtered alignment to infer the phylogenetic relationships of our sequences with the **heuristic** tree-searching method of **RAxML** using the **Maximum Likelihood (ML)** optimality criterion.

While we already discussed about some of the phylogenetic methods as well as evolutionary models during our lecture, we did not discuss about the incorporation of additional parameters, such as **invariant sites** (conservation) or variations in the substitution rates across sites. For example, **initiation codons (typically, ATG)** may not be free to vary at all. Therefore, we should make such sites invariant.

  On the other hand, we know that positions on the interior of a protein evolve more slowly that surface residues, therefore substitutions have **varying rates** depending on the position of these residues. In the case of variation, modeling these varying rates is computationally consuming, thus a well-behaved, mathematically tractable **gamma distribution** is used for modeling these rate variations.

1. Now we will starting doing some ML phylogenetics, but **which model should we use?** Well, ML has the advantage that we can use the **log-likelihood** to infer which evolutionary model is more suitable for the alignment being used. This is implemented in the **ModelTest-NG** program and can be used as follows:


In [ ]:
!modeltest-ng -i aligned_trimmed_final.fasta -d aa

The result will look very complex and unreadable. However, the models are selected according to **BIC** and **AICc**, which determine the suitability of a given model if it best-minimize **-lnL**. Here, **lnL** is the log-likelihood, with its negative value used as the minimization target during model selection. **In our particular case, the best model turned out as LG+G4**, or the **Le-Gascuel model** with **Gamma** variation.

2. Now, we can use our final MSA to perform our first phylogenetic analysis using **RAxML-NG**

In [ ]:
#RAxML does not like spaces in the fasta headers
#Keeping only IDs here
!awk '/^>/ {$0=$1} 1' aligned_trimmed_final.fasta > aligned_trimmed_raxml.fasta

#Now we run RAxML
!raxml-ng --msa aligned_trimmed_raxml.fasta --model LG+G4 --prefix T1 --threads 2

3. We can again use biopython to see the results of our phylogenetic inference using the _Phylo_ tool.

In [ ]:
from Bio import Phylo
tree = Phylo.read("T1.raxml.bestTree", "newick")
Phylo.draw(tree)

4. Now we need to test the reliability (reproducibility) of our phylogenetic tree using the **boostrapping method** that we discussed during our Lectures. Remember that this is not part of the tree construction method, but a phylogeny test.

  While usually 1000-2000 bootstrap replicates are suggested for determining the confidence of the phylogenetic tree, RAxML also has a so-called **bootstopping** method that determines how many bootstrap replicates are required to obtain stable support values. However, for a speedy tutorial, we have requested 100 boostrap replicates only.

In [ ]:
!raxml-ng --bootstrap --msa T1.raxml.rba --model LG+G4 --prefix T2 --threads 2 --bs-tree 100

**💡 HINT:** You can still check the convergence of the boostrapping test afterwards by executing the command below. In practice, a convergence cutoff value of 3% should be sufficient in most cases.

```
!raxml-ng --bsconverge --bs-trees T2.raxml.bootstraps --prefix Test --threads 2 --bs-cutoff 0.03
```


5. Now, we will map the support values obtained from the boostrapping test onto the best-scoring ML tree on the original MSA. Once you ran the cell code below, use the Phylo package again to show the phylogenetic tree with its bootstrapping values.

In [ ]:
!raxml-ng --support --tree T1.raxml.bestTree --bs-trees T2.raxml.bootstraps --prefix T3 --threads 2

In [ ]:
from Bio import Phylo
tree = Phylo.read("T3.raxml.support", "newick")
Phylo.draw(tree)

6. With this, we are done with inferring our phylogenetic trees. Also, we can get and download the **raxml.bestTree** file (which is the tree file) an see it in the visualizing website [**iTOL**](https://itol.embl.de/). 

  At the top of the iTOL website, there is an **UPLOAD** button. We can press the button and upload the besttree file to nicely draw our phylogenetic tree.

7. We will now **use the ML tree** to obtain an estimation of the **ancestral sequence** for the FoxP clade. Since we already have the MSA and the ML tree, and the ML tree is a result of the probability of each position in the alignment, we do not need anything extra to obtain the ancestral sequences at each of the internal nodes. Just use the code below.


In [ ]:
!raxml-ng --ancestral --msa aligned_trimmed_raxml.fasta --tree T1.raxml.bestTree --model LG+G4 --prefix ASR

8. Now, we will visualize the resulting **raxml.ancestralTree** file in which the names of each ancestral node in the tree. We will peak one of these nodes to print out its sequence, which is contained in the **raxml.ancestralStates**.

  The DNA encoding these protein sequences are usually synthesized by researchers to evaluate their structure and function and allowing inference of the cellular, environmental and functional contexts of ancestral and extant organisms.

In [ ]:
#Here, se the biopython Phylo package again!
from Bio import Phylo
tree = Phylo.read("ASR.raxml.ancestralTree", "newick")
Phylo.draw(tree)

In [ ]:
#Here we are printing the sequence of Node12 as our ancestral candidate for further analysis
!grep "Node12" ASR.raxml.ancestralStates

9. It is always required to check that the residue assigned by ML analysis to each position in our ancestral sequence is the most probable one. This per-position information for all nodes is contained in the **raxml.ancestralProbs** file. We are loading this information below using **pandas**.

In [ ]:
# Read Text Files with Pandas using read_csv()
  
# importing pandas
import pandas as pd

# enabling a higher number of columns on Colab
from google.colab.data_table import DataTable
DataTable.max_columns = 30

# read text file into pandas DataFrame
df = pd.read_csv("ASR.raxml.ancestralProbs", delim_whitespace=True)
  
# display DataFrame
df

# Part IV - Backing up your files

1. If you want to download your produced files, execute the code below. A compressed .tar.gz file will be generated and automatically downloaded into your computer (unless you have an ad-blocker, for which you will have to manually download it).

In [ ]:
#Compressing all files into a .tar.gz file
!tar -czf D2-tutorial-01.tar.gz *

In [ ]:
from google.colab import files
files.download("/content/D2-tutorial-01.tar.gz")

2. Alternatively, you can transfer the files directly to your Google Drive as shown below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
from pathlib import Path 
backup = Path("/content/drive/MyDrive/saocarlos2022/")
if os.path.exists(backup):
  print("Sao Carlos Workshop 2022 - Backup folder already exists")
if not os.path.exists(backup):
  os.mkdir(backup)
  print("Sao Carlos Workshop 2022 - Backup folder did not exists and was succesfully created")

#Backing up
shutil.copy(str('/content/D2-tutorial-01.tar.gz'), str(backup/'D2-tutorial-01.tar.gz'))
print("Day 2 - Tutorial 1 files successfully backed up!")